# DMG2 Assignment : Problem 5

In [17]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.neighbors import KernelDensity
from sklearn.mixture import BayesianGaussianMixture

In [2]:
DATA_DIR = '/home/jishnu/Documents/ISB/Term3/dmg2/assignments/hw_assignment1/dmg2/datasets/mnist'

In [3]:
train = pd.DataFrame(columns=['V{}'.format(i) for i in range(1,785)] + ['label'])
test = pd.DataFrame(columns=['V{}'.format(i) for i in range(1,785)] + ['label'])
for num in range(10):
    # Consolidating training data
    temp_train = pd.read_csv(os.path.join(DATA_DIR,'train{0}.csv'.format(num)),usecols=['V{}'.format(i) for i in range(1,785)])
    temp_train['label'] = num
    train = train.append(temp_train,ignore_index=True)
    # Consolidating test data
    temp_test = pd.read_csv(os.path.join(DATA_DIR,'test{0}.csv'.format(num)),usecols=['V{}'.format(i) for i in range(1,785)])
    temp_test['label'] = num
    test = test.append(temp_test,ignore_index=True)

In [4]:
train.shape

(36470, 785)

In [5]:
test.shape

(24190, 785)

In [6]:
train[train.isnull().any(axis=1)].groupby(by='label')['label'].value_counts()

label  label
4      4         46
5      5        299
6      6          1
8      8         42
Name: label, dtype: int64

In [7]:
test[test.isnull().any(axis=1)].groupby(by='label')['label'].value_counts()

label  label
4      4         35
5      5        203
6      6          4
8      8         30
Name: label, dtype: int64

**There are missing values in both the training and test data. Shown above is the count of rows with missing values, and the associated labels.**

In [8]:
train.groupby(by='label')['label'].value_counts()

label  label
0      0        3567
1      1        4034
2      2        3582
3      3        3677
4      4        3567
5      5        3567
6      6        3567
7      7        3763
8      8        3567
9      9        3579
Name: label, dtype: int64

In [9]:
test.groupby(by='label')['label'].value_counts()

label  label
0      0        2356
1      1        2708
2      2        2376
3      3        2454
4      4        2356
5      5        2356
6      6        2356
7      7        2502
8      8        2356
9      9        2370
Name: label, dtype: int64

Considering the number of complete data for each label, we can safely remove the rows with missing values for our analysis.

In [10]:
train = train.dropna()
test = test.dropna()

In [11]:
train.isnull().values.any()

False

In [12]:
test.isnull().values.any()

False

There are no missing values in the training and test data now

In [13]:
X_train = train.iloc[:,:784]
Y_train = train.iloc[:,784]

X_test = test.iloc[:,:784]
Y_test = test.iloc[:,784]

In [14]:
# Standardizing feature values
X_train = StandardScaler().fit_transform(X_train)
X_test = StandardScaler().fit_transform(X_test)

## Applying PCA

In [15]:
# Applying PCA
pc = PCA(n_components=9).fit_transform(X_train)
d1_train = pd.DataFrame(data=pc,columns=['pc{0}'.format(i) for i in range(1,10)])
d1_train['label'] = Y_train.values
d1_train.head(5)

pc = PCA(n_components=9).fit_transform(X_test)
d1_test = pd.DataFrame(data=pc,columns=['pc{0}'.format(i) for i in range(1,10)])
d1_test['label'] = Y_test.values
d1_test.head(5)

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,label
0,1.751816,-6.389704,-2.021337,-2.694470,-6.433756,1.021180,-0.539149,5.252539,3.618129,0
1,5.884330,-7.690858,-2.391167,0.260822,-4.928183,-0.383591,0.309707,6.505403,2.919571,0
2,16.381092,5.663324,-1.865806,-3.369597,4.354967,-3.093111,-5.620045,-3.819121,2.455883,0
3,12.814334,-7.638295,-4.434511,-7.660479,-0.548478,-1.786884,-0.081040,1.794377,0.516507,0
4,11.123260,7.252391,5.006899,-0.331047,15.012045,-0.077766,-1.006336,-2.364109,-0.208069,0


## Applying Fisher LDA

In [16]:
fisher = LinearDiscriminantAnalysis(n_components=9).fit_transform(X_train,Y_train.astype('int'))
d2_train = pd.DataFrame(data=fisher,columns=['f{0}'.format(i) for i in range(1,10)])
d2_train['label'] = Y_train.values
d2_train.head(5)

fisher = LinearDiscriminantAnalysis(n_components=9).fit_transform(X_test,Y_test.astype('int'))
d2_test = pd.DataFrame(data=fisher,columns=['f{0}'.format(i) for i in range(1,10)])
d2_test['label'] = Y_test.values
d2_test.head(5)

/home/jishnu/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/home/jishnu/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/home/jishnu/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


,f1,f2,f3,f4,f5,f6,f7,f8,f9,label
0,-2.833687,-1.079133,-0.845838,-0.764987,-0.484096,0.313129,-0.955681,-0.997196,1.076134,0
1,-3.849851,-3.435085,-1.935596,0.478846,-2.420500,-0.475621,0.572627,-0.989022,0.779083,0
2,-3.350865,-4.500847,-3.900589,-1.086583,-2.654538,-1.739049,-1.565408,-0.213139,-0.024020,0
3,-3.283216,-2.503597,-3.138834,-0.103059,-1.582149,-0.295842,1.330582,-3.239352,1.473339,0
4,-2.231976,-1.552025,-3.889057,-0.017441,-0.577388,-2.299944,-0.684894,0.945457,0.404140,0


## Building Bayesian Classifier on D1 Dataset

In [18]:
d1_train_X = d1_train.iloc[:,:9]
d1_train_Y = d1_train.iloc[:,9].astype('int')

d1_test_X = d1_test.iloc[:,:9]
d1_test_Y = d1_test.iloc[:,9].astype('int')

d2_train_X = d2_train.iloc[:,:9]
d2_train_Y = d2_train.iloc[:,9].astype('int')

d2_test_X = d2_test.iloc[:,:9]
d2_test_Y = d2_test.iloc[:,9].astype('int')

In [44]:
bayes_clf_acc = pd.DataFrame(columns=['Dataset','Covariance','Test Accuracy'])

In [45]:
bayesian_d1_diag = BayesianGaussianMixture(n_components=10,covariance_type='diag',max_iter=100000)
bayesian_d1_diag = bayesian_d1_diag.fit(d1_train_X,d1_train_Y)
bayes_d1_diag_df = pd.DataFrame({'actual':d1_test_Y,'predicted':bayesian_d1_diag.predict(d1_test_X)})
bayes_clf_acc = bayes_clf_acc.append({'Dataset':'D1','Covariance':'diag','Test Accuracy':np.round(bayes_d1_diag_df.loc[bayes_d1_diag_df['actual'] == bayes_d1_diag_df['predicted']].shape[0]/bayes_d1_diag_df.shape[0],4)},ignore_index=True)

In [46]:
bayesian_d1_full = BayesianGaussianMixture(n_components=10,covariance_type='full',max_iter=10000)
bayesian_d1_full = bayesian_d1_full.fit(d1_train_X,d1_train_Y)
bayes_d1_full_df = pd.DataFrame({'actual':d1_test_Y,'predicted':bayesian_d1_full.predict(d1_test_X)})
bayes_clf_acc = bayes_clf_acc.append({'Dataset':'D1','Covariance':'full','Test Accuracy':np.round(bayes_d1_full_df.loc[bayes_d1_full_df['actual'] == bayes_d1_full_df['predicted']].shape[0]/bayes_d1_full_df.shape[0],4)},ignore_index=True)

In [47]:
bayesian_d2_diag = BayesianGaussianMixture(n_components=10,covariance_type='diag',max_iter=100000)
bayesian_d2_diag = bayesian_d2_diag.fit(d2_train_X,d2_train_Y)
bayes_d2_diag_df = pd.DataFrame({'actual':d2_test_Y,'predicted':bayesian_d2_diag.predict(d2_test_X)})
bayes_clf_acc = bayes_clf_acc.append({'Dataset':'D2','Covariance':'diag','Test Accuracy':np.round(bayes_d2_diag_df.loc[bayes_d2_diag_df['actual'] == bayes_d2_diag_df['predicted']].shape[0]/bayes_d2_diag_df.shape[0],4)},ignore_index=True)

In [48]:
bayesian_d2_full = BayesianGaussianMixture(n_components=10,covariance_type='full',max_iter=100000)
bayesian_d2_full = bayesian_d2_full.fit(d2_train_X,d2_train_Y)
bayes_d2_full_df = pd.DataFrame({'actual':d2_test_Y,'predicted':bayesian_d2_full.predict(d2_test_X)})
bayes_clf_acc = bayes_clf_acc.append({'Dataset':'D2','Covariance':'full','Test Accuracy':np.round(bayes_d2_full_df.loc[bayes_d2_full_df['actual'] == bayes_d2_full_df['predicted']].shape[0]/bayes_d2_full_df.shape[0],4)},ignore_index=True)

In [49]:
bayes_clf_acc

,Dataset,Covariance,Test Accuracy
0,D1,diag,0.1842
1,D1,full,0.0687
2,D2,diag,0.2298
3,D2,full,0.0943


The test accuracies for the four classifier show that, classifiers using Fisher projections dataset has higher accuracies when compared to the PCA projected dataset.

It is also seen that classifiers with diagonal covariance matrix have better accuracies when compared to those full covariance matrix, for this dataset.